In [1]:
import os
import torch
from img2mol.inference import *

/home/jh-li/mambaforge/envs/img2mol-env/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Local CDDD installation has not been found.


In [2]:
# load model
device = "cuda:0" if torch.cuda.is_available() else "cpu"
img2mol = Img2MolInference(
    model_ckpt=os.path.join("model", "model.ckpt"),
    device=device,
)

Initializing Img2Mol Model with random weights.
Loading checkpoint: model/model.ckpt
Setting to `self.eval()`-mode.
Sending model to `cpu` device.
Succesfully created Img2Mol Inference class.


In [3]:
# infer with original model
dummy_input = torch.rand(1, 1, 224, 224)
dummy_output = img2mol.model(dummy_input)

In [4]:
# create workdir
workdir = "img2mol_pnnx"
os.makedirs(workdir, exist_ok=True)

# export model with torch.jit.trace
img2mol_trace = torch.jit.trace(img2mol.model, dummy_input)
torch.jit.save(img2mol_trace, os.path.join(workdir, "img2mol.pt"))

/home/jh-li/mambaforge/envs/img2mol-env/lib/python3.6/site-packages/torch/_jit_internal.py:505: LightningDeprecationWarning: The `LightningModule.loaded_optimizer_states_dict` property is deprecated in v1.4 and will be removed in v1.6.
  item = getattr(mod, name)
/home/jh-li/mambaforge/envs/img2mol-env/lib/python3.6/site-packages/torch/_jit_internal.py:505: LightningDeprecationWarning: The `LightningModule.model_size` property was deprecated in v1.5 and will be removed in v1.7. Please use the `pytorch_lightning.utilities.memory.get_model_size_mb`.
  item = getattr(mod, name)


In [5]:
# use pnnx to convert model
! cd img2mol_pnnx && /opt/pnnx-20231010-ubuntu/pnnx img2mol.pt "inputshape=[1,1,224,224]"

pnnxparam = img2mol.pnnx.param
pnnxbin = img2mol.pnnx.bin
pnnxpy = img2mol_pnnx.py
pnnxonnx = img2mol.pnnx.onnx
ncnnparam = img2mol.ncnn.param
ncnnbin = img2mol.ncnn.bin
ncnnpy = img2mol_ncnn.py
fp16 = 1
optlevel = 2
device = cpu
inputshape = [1,1,224,224]f32
inputshape2 = 
customop = 
moduleop = 
############# pass_level0

----------------

[W NNPACK.cpp:64] Could not initialize NNPACK! Reason: Unsupported hardware.
############# pass_level1
############# pass_level2
############# pass_level3
############# pass_level4
############# pass_level5
############# pass_ncnn


In [6]:
os.chdir(workdir)

import img2mol_pnnx

# load pnnx model
img2mol_pnnx_model = img2mol_pnnx.Model()
img2mol_pnnx_model.eval()

os.chdir("..")

In [7]:
# compare results
dummy_output_pnnx = img2mol_pnnx_model(dummy_input)
torch.cosine_similarity(dummy_output, dummy_output_pnnx)

tensor([1.], grad_fn=<DivBackward0>)